<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - Bias nDCG</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender
from lenskit.algorithms.basic import Bias
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer(item,user,damping):
    all_recs = []
    test_data = []
    
    bias = Bias(items=item, users=user, damping=damping) # define algorithm
    
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('Bias', bias, train, test))
    
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.ndcg)
    results = rla.compute(all_recs, test_data)
                          
    return results["ndcg"].mean()

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    user = params['user']
    item = params['item']
    damping = params['damping']
    metric = model_trainer(item,user,damping)
    print([item,user,damping,metric],',')
    return {'loss': -metric, 'status': STATUS_OK }

space={'item': hp.choice('item', ['True', 'False']),
       'user': hp.choice('user', ['True', 'False']),
       'damping': hp.uniform('damping', 0, 1)}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

['False', 'True', 0.6825041547340321, 0.08132031441797694]                                                             
,                                                                                                                      
['False', 'False', 0.6154253424891756, 0.08168400822417445]                                                            
,                                                                                                                      
['True', 'False', 0.9953943039772094, 0.08834846845196243]                                                             
,                                                                                                                      
['False', 'False', 0.48424981649207055, 0.07896742576286479]                                                           
,                                                                                                                      
['False', 'True', 0.28647741701515916, 0

['True', 'False', 0.8623848955071406, 0.08448865530819501]                                                             
,                                                                                                                      
['True', 'False', 0.8815465627810082, 0.08387466126618738]                                                             
,                                                                                                                      
['True', 'False', 0.6913098808330707, 0.08226525604773736]                                                             
,                                                                                                                      
['True', 'False', 0.9332858489927428, 0.08521190646978039]                                                             
,                                                                                                                      
['True', 'False', 0.8224805599498098, 0.

In [4]:
array = [['False', 'True', 0.6825041547340321, 0.08132031441797694]                                                             
,                                                                                                                      
['False', 'False', 0.6154253424891756, 0.08168400822417445]                                                            
,                                                                                                                      
['True', 'False', 0.9953943039772094, 0.08834846845196243]                                                             
,                                                                                                                      
['False', 'False', 0.48424981649207055, 0.07896742576286479]                                                           
,                                                                                                                      
['False', 'True', 0.28647741701515916, 0.07772182897519855]                                                            
,                                                                                                                      
['True', 'True', 0.6616826947291119, 0.08196201488172801]                                                              
,                                                                                                                      
['True', 'True', 0.3100872755119929, 0.0775338308917796]                                                               
,                                                                                                                      
['False', 'False', 0.4082777022590103, 0.07940197365339177]                                                            
,                                                                                                                      
['False', 'False', 0.8189519908661689, 0.08445582130526078]                                                            
,                                                                                                                      
['False', 'False', 0.27262957089082507, 0.07962281931750574]                                                           
,                                                                                                                      
['False', 'True', 0.7164634222404208, 0.08320893897490082]                                                             
,                                                                                                                      
['True', 'False', 0.04504755056365617, 0.07355284513999526]                                                            
,                                                                                                                      
['False', 'False', 0.9577332285106652, 0.0853515757473364]                                                             
,                                                                                                                      
['True', 'False', 0.5030106835909605, 0.08045854910927461]                                                             
,                                                                                                                      
['True', 'True', 0.8815670478447203, 0.08443121480091008]                                                              
,                                                                                                                      
['False', 'True', 0.035910176625931545, 0.07376814819866688]                                                           
,                                                                                                                      
['False', 'True', 0.8904563849083211, 0.08321223553914922]                                                             
,                                                                                                                      
['False', 'True', 0.8201126064736213, 0.08532582539345744]                                                             
,                                                                                                                      
['True', 'True', 0.5710365349129892, 0.0797900037778603]                                                               
,                                                                                                                      
['False', 'False', 0.7281851751039288, 0.07938738572676837]                                                            
,                                                                                                                      
['True', 'False', 0.9990759520531, 0.08628802905917968]                                                                
,                                                                                                                      
['True', 'False', 0.9966835315993852, 0.08425319418477632]                                                             
,                                                                                                                      
['True', 'False', 0.959562177736867, 0.08730907759892295]                                                              
,                                                                                                                      
['True', 'False', 0.795783774025999, 0.08285070924606762]                                                              
,                                                                                                                      
['True', 'False', 0.9197843535932013, 0.08391648052445982]                                                             
,                                                                                                                      
['True', 'False', 0.169839988297792, 0.0740201917890353]                                                               
,                                                                                                                      
['True', 'False', 0.9934091560966152, 0.0903277367821774]                                                              
,                                                                                                                      
['True', 'False', 0.7853966238800683, 0.08431737333006686]                                                             
,                                                                                                                      
['True', 'False', 0.9979409187745361, 0.08633765269592399]                                                             
,                                                                                                                      
['True', 'False', 0.8716228313175131, 0.08246046981149495]                                                             
,                                                                                                                      
['True', 'False', 0.6341988860758851, 0.0808808728721531]                                                              
,                                                                                                                      
['True', 'False', 0.5639527151625467, 0.0801947299991279]                                                              
,                                                                                                                      
['True', 'False', 0.7362431106627909, 0.08195314175410602]                                                             
,                                                                                                                      
['True', 'False', 0.10117791210541227, 0.07582370577584926]                                                            
,                                                                                                                      
['True', 'False', 0.38791908989941537, 0.07868053787940191]                                                            
,                                                                                                                      
['True', 'False', 0.9494977754639218, 0.08486777163310263]                                                             
,                                                                                                                      
['True', 'False', 0.6653364229614777, 0.08210925670642484]                                                             
,                                                                                                                      
['True', 'False', 0.8439511221446718, 0.08435922697437603]                                                             
,                                                                                                                      
['True', 'False', 0.45878478242752435, 0.07771615502941162]                                                            
,                                                                                                                      
['True', 'True', 0.22055963795697386, 0.07792813180353202]                                                             
,                                                                                                                      
['True', 'False', 0.3504733375837792, 0.08020879876081835]                                                             
,                                                                                                                      
['True', 'False', 0.585298235683552, 0.08045459772449187]                                                              
,                                                                                                                      
['False', 'True', 0.694304302982448, 0.0834939626186358]                                                               
,                                                                                                                      
['True', 'False', 0.7531940402836576, 0.08144226865393044]                                                             
,                                                                                                                      
['False', 'False', 0.9176776831128194, 0.08564910781082366]                                                            
,                                                                                                                      
['True', 'True', 0.479947055098103, 0.07709344204002232]                                                               
,                                                                                                                      
['True', 'False', 0.8537956683057968, 0.08339873315336739]                                                             
,                                                                                                                      
['False', 'True', 0.5168381178769756, 0.07953408548999866]                                                             
,                                                                                                                      
['True', 'False', 0.7827556747887712, 0.08272943567110519]                                                             
,                                                                                                                      
['False', 'False', 0.616023813540544, 0.08050454956915913]                                                             
,                                                                                                                      
['True', 'True', 0.9592411713835557, 0.08675680984797855]                                                              
,                                                                                                                      
['True', 'False', 0.9114558601866224, 0.0848396736682629]                                                              
,                                                                                                                      
['False', 'False', 0.2568059912977746, 0.07953415325100063]                                                            
,                                                                                                                      
['True', 'True', 0.987101193207044, 0.08675801434238593]                                                               
,                                                                                                                      
['True', 'False', 0.4392458678159449, 0.08037525131784562]                                                             
,                                                                                                                      
['False', 'False', 0.7106108902442037, 0.08146713018035218]                                                            
,                                                                                                                      
['True', 'False', 0.8431090918287978, 0.08637773303830369]                                                             
,                                                                                                                      
['True', 'True', 0.07543913068519892, 0.07779090206811404]                                                             
,                                                                                                                      
['True', 'False', 0.5475697820162218, 0.07634570561819941]                                                             
,                                                                                                                      
['False', 'False', 0.6402567707515544, 0.08131073462996893]                                                            
,                                                                                                                      
['True', 'False', 0.7663496819006473, 0.08478815394350511]                                                             
,                                                                                                                      
['True', 'True', 0.8079205020324913, 0.08692365997615892]                                                              
,                                                                                                                      
['True', 'False', 0.15932984843871506, 0.07797700063920786]                                                            
,                                                                                                                      
['False', 'False', 0.3073754417544493, 0.07841913729267815]                                                            
,                                                                                                                      
['True', 'False', 0.8996782170016496, 0.08290522932515096]                                                             
,                                                                                                                      
['True', 'False', 0.9487106163191948, 0.08436189048250596]                                                             
,                                                                                                                      
['True', 'False', 0.973951258750851, 0.08483763241754443]                                                              
,                                                                                                                      
['True', 'False', 0.9438503798813541, 0.0866442653798153]                                                              
,                                                                                                                      
['True', 'False', 0.8623848955071406, 0.08448865530819501]                                                             
,                                                                                                                      
['True', 'False', 0.8815465627810082, 0.08387466126618738]                                                             
,                                                                                                                      
['True', 'False', 0.6913098808330707, 0.08226525604773736]                                                             
,                                                                                                                      
['True', 'False', 0.9332858489927428, 0.08521190646978039]                                                             
,                                                                                                                      
['True', 'False', 0.8224805599498098, 0.08547790033762329]                                                             
,                                                                                                                      
['True', 'False', 0.9850420652684447, 0.08588028384481124]                                                             
,                                                                                                                      
['True', 'False', 0.8298105189839169, 0.08548461291096035]                                                             
,                                                                                                                      
['True', 'False', 0.7533467012271684, 0.08276838186881186]                                                             
,                                                                                                                      
['True', 'False', 0.8986633803180266, 0.08318876353466093]                                                             
,                                                                                                                      
['True', 'False', 0.6651145743973832, 0.08298530719290555]                                                             
,                                                                                                                      
['True', 'True', 0.7962325692297747, 0.08208560910551707]                                                              
,                                                                                                                      
['True', 'False', 0.7256131023739087, 0.08373892457626397]                                                             
,                                                                                                                      
['False', 'False', 0.967695662687602, 0.08556717338327377]                                                             
,                                                                                                                      
['True', 'False', 0.8705093986859337, 0.08413005832921995]                                                             
,                                                                                                                      
['True', 'True', 0.012669676616197278, 0.07509483891662369]                                                            
,                                                                                                                      
['True', 'False', 0.39715039944548447, 0.07771674512434043]                                                            
,                                                                                                                      
['True', 'False', 0.7721955313493805, 0.08533563401457851]                                                             
,                                                                                                                      
['False', 'False', 0.9216160035310743, 0.08417563782958802]                                                            
,                                                                                                                      
['True', 'False', 0.9820923256160408, 0.08489446757565457]                                                             
,                                                                                                                      
['True', 'True', 0.5895904141719486, 0.08173217364957705]                                                              
,                                                                                                                      
['True', 'False', 0.533597765014564, 0.07782809986800422]                                                              
,                                                                                                                      
['False', 'False', 0.9337581471516493, 0.08360292494735302]                                                            
,                                                                                                                      
['True', 'False', 0.6373702574607679, 0.08291482087659013]                                                             
,                                                                                                                      
['True', 'True', 0.8408737977149547, 0.08468218107422931]                                                              
,                                                                                                                      
['True', 'False', 0.36780896421264764, 0.07844455592454277]                                                            
,                                                                                                                      
['False', 'False', 0.44202515590517577, 0.07904144529043529]                                                           
,                                                                                                                      
['True', 'False', 0.9950089318927394, 0.08263787092039736]                                                             
,                                                                                                                      
['True', 'False', 0.7078333437944757, 0.08133090223443862]                                                             
,                                                                                                                      
['True', 'True', 0.7434242103349183, 0.08368055909288553]                                                              
,                                                                                                                      
['True', 'False', 0.9022819566438501, 0.08222074999225071]                                                             
,                                                                                                                      
['False', 'False', 0.20573902247891318, 0.07711814156781394]                                                           
,                                                                                                                      
['True', 'False', 0.8108312088270434, 0.08290175200433855] ]

In [5]:
metric = pd.DataFrame(array)
metric.columns = ['items','users','damping','nDCG']
metric.head(5)

,items,users,damping,nDCG
0,False,True,0.682504,0.081320
1,False,False,0.615425,0.081684
2,True,False,0.995394,0.088348
3,False,False,0.484250,0.078967
4,False,True,0.286477,0.077722


In [6]:
metric.sort_values(by=['nDCG'],ascending=False).head(10)

,items,users,damping,nDCG
26,True,False,0.993409,0.090328
2,True,False,0.995394,0.088348
22,True,False,0.959562,0.087309
61,True,True,0.807921,0.086924
53,True,True,0.987101,0.086758
50,True,True,0.959241,0.086757
67,True,False,0.943850,0.086644
56,True,False,0.843109,0.086378
28,True,False,0.997941,0.086338
20,True,False,0.999076,0.086288


In [7]:
metric.to_csv('metric_ndcg.csv', index=False)